In [18]:
import os
import subprocess
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import tensorflow as tf
from tensorflow.keras.models import load_model
import imageio

_ = load_dotenv(find_dotenv())

In [3]:
def ConnectToAzure():
    """
    desc:
        Function connects to langchain AzureOpenAI
    return: 
        model of llm
    """
    ## Keys ##
    OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
    OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
    OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

    model = AzureChatOpenAI(
        openai_api_base=OPENAI_API_BASE,
        openai_api_version=OPENAI_API_VERSION,
        azure_deployment=DEPLOYMENT_NAME,
        openai_api_key=OPENAI_API_KEY,
        openai_api_type=OPENAI_API_TYPE,
    )
    return model

In [7]:
def ConversationInput():
    _DEFAULT_TEMPLATE = """
    You are an expert in art and artists.
    The human has provided an image of an art piece, 
    which has been analyzed and classified with percentages indicating the likelihood of it being by specific artists. 
    You will receive these percentages are required to comment on the results.
    
    Current conversation:
    New human question: {input}
    Response:"""

    prompt = PromptTemplate(
        input_variables=["input"], template=_DEFAULT_TEMPLATE
    )

    conversation = LLMChain(
    llm=ConnectToAzure(),
    prompt=prompt,
    verbose=False,
    )
    return conversation

In [12]:
def preprocess_image(img):
    img = cv2.resize(img, (train_input_shape[0], train_input_shape[1]))
    img = image.img_to_array(img)
    img /= 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [27]:
model = load_model('model/Artist_Resnet_model.h5')

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_2'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
img = imageio.imread(file_path)
            preprocessed_img = preprocess_image(img)
            prediction = model.predict(preprocessed_img)

In [11]:
conversation = ConversationInput()

response = conversation.predict(input = "Da Vinci: 30% ||| Van Gogh: 50% ||| Picasso: 20%")
response

C:\Users\Ahmed\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://eslsca-openai.openai.azure.com/ to https://eslsca-openai.openai.azure.com/openai.
  warnings.warn(
C:\Users\Ahmed\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\Ahmed\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of

"Based on the percentages provided, there is a 50% likelihood that the art piece was created by Van Gogh, followed by a 30% likelihood of it being by Da Vinci, and finally a 20% likelihood of it being by Picasso. These percentages indicate the level of resemblance or similarity between the analyzed art piece and the artistic styles of these artists.\n\nIt is important to note that these percentages are not definitive and should be interpreted with caution. Artistic style can vary greatly, and there may be other factors that could influence the analysis. Additionally, the analysis is based on patterns and characteristics commonly associated with the artists, but it does not take into account individual variations or potential forgeries.\n\nFurther investigation and analysis would be necessary to make a more conclusive determination about the artist behind the art piece. This could involve examining the techniques, subject matter, historical context, and other aspects that are characteri